In [3]:
!pip install mecab-ko-msvc mecab-ko-dic-msvc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 MB 8.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score

In [5]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
df = pd.read_csv('/content/drive/MyDrive/esg 프로젝트/csv/news_토스_라벨링.csv')
df

,Unnamed: 0,기사날짜,기사제목,네이버 뉴스 url,기사내용,감정분석
0,0.0,2022.04.13.,"캐치테이블, 알토스벤처스 외 300억 규모 시리즈C 투자 유치",https://n.news.naver.com/mnews/article/009/000...,실시간 레스토랑 예약 플랫폼 ‘캐치테이블’ 운영사 와드(용태순 대표)가 300억 규...,중립
1,1.0,2022.04.05.,까다로운 주담대…인뱅 문턱은 낮아졌다,https://n.news.naver.com/mnews/article/003/001...,기사내용 요약비대면·낮은 금리에 인터넷은행 대출 '인기'한도 확대·상품 출시 등 선...,긍정
2,2.0,2022.04.04.,“9 to 4 아니어도 대출 OK”…은행 영업시간 외 대출 44%,https://n.news.naver.com/mnews/article/016/000...,"핀다, 대출 실행 데이터 분석오후 4시~5시 이용 가장 많아주말 대출도 전년대비 7...",긍정
3,3.0,2022.04.08.,꺾이지 않는 나쁜 인플레에 결국 '불황' 오나 [글로벌체크],https://n.news.naver.com/mnews/article/011/000...,[김연하의 글로벌체크]2월 美 CPI 7.9%로 40년만 최고치3월도 8.4% 전망...,부정
4,4.0,2022.04.12.,"리서치알음 ""웹케시, 횡령사고 막는 국내 유일 B2B핀테크…회원수 증가 매출 ↑""",https://n.news.naver.com/mnews/article/014/000...,[파이낸셜뉴스] 리서치알음은 12일 웹케시에 대해 최근 불거진 횡령사건에 대비할 수...,중립
...,...,...,...,...,...,...
2394,45.0,2023.03.26.,글로벌 간편결제 'GLN' 국내외 금융·핀테크 플랫폼 대연합,https://n.news.naver.com/mnews/article/030/000...,국민銀 스타뱅킹앱에 기능 연동카카오페이와 연내 서비스 출시해외 선불충전 '띳머니' ...,긍정
2395,46.0,2023.03.22.,"한국투자증권, 카뱅·토뱅과 토큰증권협의체 결성",https://n.news.naver.com/mnews/article/366/000...,"한국투자증권 제공 한국투자증권이 인터넷 전문은행 카카오뱅크, 토스뱅크와...",긍정
2396,47.0,2023.03.24.,이주의 새책,https://n.news.naver.com/mnews/article/009/000...,"악티움해전배리 스트라우스 지음, 이종인 옮김, 2만8000원악티움해전은 옥타비아누스...",중립
2397,48.0,2023.03.24.,‘편리성’ 인터넷銀-‘자금력’ 지방銀 손잡고 공동대출 추진,https://n.news.naver.com/mnews/article/020/000...,"시중은행 과점체제 개혁 일환토스, 광주은행과 실무협약 끝내예금금리 한번에 비교후 가...",긍정


In [110]:
df = df.rename(columns={'기사날짜': 'date', '기사내용': 'content'})

In [112]:
# 만약 시간 데이터 열이 문자열 형식이라면, 날짜/시간 형식으로 변환
df['date'] = pd.to_datetime(df['date'])

# 'time_column'을 기준으로 오름차순으로 정렬
df = df.sort_values(by='date', ascending=True)

In [44]:
def mecab_tokenizer(text):
    # 입력이 문자열이 아닐 경우 빈 리스트 반환
    if not isinstance(text, str):
        return []

    mecab = MeCab.Tagger()
    parsed = mecab.parse(text)
    words = []
    for line in parsed.split('\n'):
        if '\t' in line:
            word, tag = line.split('\t')[0], line.split('\t')[1].split(',')[0]
            if tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG'] and len(word) > 1:
                words.append(word)
    return words

In [45]:
def calculate_similarity(text1, text2):
    tokens1 = set(mecab_tokenizer(text1))
    tokens2 = set(mecab_tokenizer(text2))
    if not tokens1 or not tokens2:
        return 0
    return len(tokens1.intersection(tokens2)) / len(tokens1)

In [46]:
def remove_similar_texts(df, threshold=0.8):
    # 가장 최근 날짜로부터 일주일 전의 날짜를 계산
    one_week_ago = df['date'].max() - pd.Timedelta(days=7)

    # 일주일 전부터 현재까지의 데이터를 필터링
    recent_df = df[df['date'] >= one_week_ago]

    remove_indices = set()

    # 필터링된 데이터프레임에 대해 유사성 검사 수행
    for i, text1 in enumerate(recent_df['content']):
        for j, text2 in enumerate(recent_df['content']):
            if i != j and i not in remove_indices and j not in remove_indices:
                similarity = calculate_similarity(text1, text2)
                if similarity >= threshold:
                    remove_indices.add(recent_df.index[j])

    # 유사한 텍스트가 포함된 행을 제거
    return df.drop(list(remove_indices))

In [12]:
#df = df.drop(df.columns[[0, -1]], axis=1)

In [ ]:
nan_rows_CONTENT = df[df['content'].isna()]
nan_rows_CONTENT

In [113]:
df= remove_similar_texts(df)

In [ ]:
df

In [114]:
# 데이터를 훈련 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['감정분석'], test_size=0.2, random_state=42)

In [115]:
# 2. 모델 훈련
model = make_pipeline(TfidfVectorizer(tokenizer=mecab_tokenizer), MultinomialNB())
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function mecab_tokenizer at 0x793a778a7d00>)),
                ('multinomialnb', MultinomialNB())])

In [116]:
# 3. 성능 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print(f'accuracy: {accuracy:.2f}')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

accuracy: 0.53
Precision: 0.38
Recall: 0.38


In [ ]:
# 모델 저장
import joblib
joblib.dump(model, '/content/multinomial_nb_model.pkl')

['/content/multinomial_nb_model.pkl']

In [ ]:
import joblib
model = joblib.load('/content/multinomial_nb_model.pkl')

In [117]:
df2 = pd.read_csv('/content/drive/MyDrive/esg 프로젝트/csv/news_토스_기사데이터.csv')

In [118]:
df2 = df2.rename(columns={'기사날짜': 'date', '기사내용': 'content'})

In [119]:
# 만약 시간 데이터 열이 문자열 형식이라면, 날짜/시간 형식으로 변환
df2['date'] = pd.to_datetime(df2['date'])

# 'time_column'을 기준으로 오름차순으로 정렬
df2 = df2.sort_values(by='date', ascending=True)
df2 = df2.dropna(subset=['content'])
df2 = df2[~df2['content'].str.contains('<네이버 기사 없음>')]

In [120]:
df2

,date,기사제목,네이버 뉴스 url,content
1919,2021-10-01,[데일리안 오늘뉴스 종합] '유동규 뇌물수수 의혹' 뇌관…민주당 당혹감 속 언급 피...,https://n.news.naver.com/mnews/article/119/000...,더불어민주당 대선 주자인 이재명 경기도지사가 지난 9월 29일 오전 서울 영등포구 ...
926,2021-10-01,"연말까지 '마통'닫은 카카오뱅크…토스는 ""억대 대출""",https://n.news.naver.com/mnews/article/422/000...,[앵커] 인터넷전문은행 카카오뱅크가 금융당국의 가계대출 억제 권고에 따라 오늘(1일...
2012,2021-10-01,[전문가에게 듣는다] 방역 당국 “코로나19 예방접종 증명서 불법 사용 시 처벌”,https://n.news.naver.com/mnews/article/056/001...,[앵커] 코로나19 백신 접종률이 오르고 단계적 일상회복 논의가 진행되면서 접종 완...
1180,2021-10-01,2%대 금리 은행 온다...토스뱅크 내주 출범,https://n.news.naver.com/mnews/article/029/000...,10월 5일 사전 신청자 대상 서비스예금·카드·대출 상품 계획 발표업계 최저 2%대...
1684,2021-10-01,[황출새] 최재형 'BTS 등골 빼먹는 文 정부' 논평에 탁현민 '지긋지긋 하다' 外,https://n.news.naver.com/mnews/article/052/000...,[관련 동영상 보기]YTN라디오(FM 94.5) [황보선의 출발 새아침]□ 방송일시...
...,...,...,...,...
62338,2023-09-30,"긴 황금연휴, 급한 택배 보내려면 '이곳'에서",https://n.news.naver.com/mnews/article/417/000...,편의점 GS25가 추석 연휴 기간 반값택배를 정상 운영한다. 모델이 추석 연휴에도 ...
61538,2023-09-30,아프간 여자배구 대표팀의 완패에 담긴 ‘압도적 희망’,https://n.news.naver.com/mnews/article/028/000...,망명자로 이뤄진 여자배구 대표팀카자흐스탄전서 0-3 완패했지만…아프가니스탄 여자배구...
61542,2023-09-30,아프간 여자배구 대표팀의 완패에 담긴 ‘압도적 희망’,https://n.news.naver.com/mnews/article/028/000...,망명자로 이뤄진 여자배구 대표팀카자흐스탄전서 0-3 완패했지만…아프가니스탄 여자배구...
62196,2023-09-30,추석에 받은 용돈…이색 예·적금은 어때?,https://n.news.naver.com/mnews/article/648/000...,인뱅 최대 4%대 고금리 예·적금…청소년층에 큰 인기시중은행들도 MZ 사로잡기…다양...


In [23]:
E_keywords = {'친환경', '물질', '기후', '발생', '요인', '화학', '배출', '생물', '온실가스', '오염', '보전', '생태', '탄소', '유해', '용수', '순환', '산림', '사용량', '부자재', '해양', '설비', '자연', '수자원', '폐수', '녹색', '사슬', '재생', '정량', '지역', '폐기', '배출권', '저탄소', '건강', '플라스틱', '인벤토리', '보호', '피해', '원료', '저감', '신재'}
S_keywords = {'위험', '규준', '모범', '공개', '참고', '문화', '의사', '요인', '체계', '개인', '안전', '공급', '발생', '제도', '효과', '피해', '가치', '소통', '파악', '구축', '리더십', '보건', '사항', '침해', '보장', '점검', '실사', '결과', '보상', '역할', '결함', '공유', '성장', '방침', '자유', '경제', '생활', '상호', '고용', '관계', '예방', '정기', '달성', '창출', '주체', '범위', '동반성', '부당', '균형', '형성', '규제', '보안', '강령', '연계', '감독', '저해', '권익', '부합', '표명', '자발', '개정', '안전성', '고충', '임금', '업무', '자율', '긍정', '증진', '이익', '정의', '특성', '투입', '근거', '유도', '적절', '해소', '조성', '인지', '뒷받침', '노사', '구조', '산업', '국내외', '근무', '수단', '남용', '관여', '건강', '차별', '기반', '수집', '전달', '협의', '유형', '인력', '대내외', '상이', '정량', '모델', '중대', '계층', '완화', '정당', '관점', '선택', '금지', '선제', '방지', '전문', '잠재', '중심', '질서', '조건', '합리', '비용', '이용', '생산', '조사', '확립', '가격', '메커니즘', '대책', '접근성', '취약', '성별', '조치', '사고', '우려', '실질', '불공정', '시행', '비율', '재해', '기업지배구조', '경영진', '변화', '채널', '자사', '절차', '부정', '행위', '외부', '가이드라인', '임직원', '부서', '구성', '사례', '프로그램', '목적', '처리', '구체', '기본', '설정', '측면', '부여'}
G_keywords = {'감사', '이사', '주주', '위원회', '지배', '모범', '규준', '사외', '기업지배구조', '총회', '선임', '규정', '중대', '내용', '관계', '직무', '재무', '후보', '기능', '경영자', '결의', '보수', '독립성', '전문', '의사', '권고', '주식', '금지', '행사', '회계', '위원', '소통', '이익', '승계', '이사회', '공개', '법령', '의결권', '보상', '임원', '보호', '사유', '환경', '지배주', '추천', '다양', '부서', '노력', '자본', '정기', '참석', '개최', '책임자', '포함', '임직원', '윤리', '대표', '설치', '의견', '투명', '제도', '행위', '공정', '권리', '변화', '정관', '마련', '정책', '회의', '유지', '이용', '영업', '반대', '보고', '자격', '보장', '타당', '제정', '영향', '매수', '제한', '안건', '일반', '판단', '개정', '과도', '사전', '보고서', '적정', '직접', '정확', '투자', '비밀', '교육', '의안', '목적', '승인', '자신', '과정', '해임', '자료', '집행', '자기', '독립', '이해관계', '상충', '합병', '분리', '내역', '담당', '방법', '침해', '발생', '합리', '자산', '설명', '활용', '의미', '관행', '근거', '회의록', '제시', '용이', '방향', '작성', '전문가', '위임', '집단', '의장', '예측', '요청', '활성', '자문', '훼손', '인수', '지급', '상정', '존중', '지정', '일치', '자기거래', '양수', '상장', '지식', '시장', '법인', '투표', '부여', '논의', '용역', '소유', '추구', '주의', '특수', '이상', '지양', '영향력', '계열사', '법률', '조성', '인과', '사내', '연간', '손해', '부당', '전원', '개별', '토의', '명문화', '상세', '보유', '공유', '신속', '수단', '가진', '접근', '전자', '적시', '기재', '장소', '방안', '우려', '보험', '투입', '실효', '원활', '외국인', '전체', '개인', '명확', '일정', '성장', '가입', '채권자', '항목'}


In [121]:
# 데이터프레임에서 유사도가 높은 텍스트 제거
df3 = remove_similar_texts(df2)

In [ ]:
df3

In [57]:
def find_most_similar_category(text):
    # 텍스트를 토큰화
    tokens = set(mecab_tokenizer(text))

    # 각 카테고리와의 유사도 계산
    E_similarity = len(E_keywords.intersection(tokens))
    S_similarity = len(S_keywords.intersection(tokens))
    G_similarity = len(G_keywords.intersection(tokens))

    # 'E' 카테고리 유사도가 3 이상인 경우 무조건 'E' 반환
    if E_similarity >= 3:
        return 'E'

    # 나머지 카테고리 중 가장 유사도가 높은 카테고리 찾기
    similarities = {'E': E_similarity, 'S': S_similarity, 'G': G_similarity}
    most_similar_category = max(similarities, key=similarities.get, default='X')

    # 유사도가 0이면 'X' 반환
    if similarities[most_similar_category] == 0:
        return 'X'
    return most_similar_category

In [27]:
def calculate_sentence_category_percentages(text):
    sentences = text.split('.')
    category_counts = {'E': 0, 'S': 0, 'G': 0}

    for sentence in sentences:
        if sentence.strip():
            category = find_most_similar_category(sentence)
            if category in category_counts:
                category_counts[category] += 1

    total_sentences = len(sentences)

    category_percentages = {}
    if total_sentences > 0:
        esg_total = 0
        for k, v in category_counts.items():
            percentage = (v / total_sentences) * 100
            category_percentages[k] = round(percentage, 2)
            esg_total += v
        category_percentages['ESG_Total'] = round((esg_total / total_sentences) * 100, 2)
    else:
        for k in category_counts:
            category_percentages[k] = 0
        category_percentages['ESG_Total'] = 0

    return category_percentages

In [34]:
def add_category_percentages_to_df(df):
    for index, row in df.iterrows():
        percentages = calculate_sentence_category_percentages(row['content'])
        df.at[index, 'ESG_Total %'] = percentages['ESG_Total']

    return df

In [29]:
def count_keywords_in_sentence(sentence, category, E_keywords, S_keywords, G_keywords):
    # 각 카테고리에 해당하는 키워드 세트를 선택합니다.
    if category == 'E':
        keywords = E_keywords
    elif category == 'S':
        keywords = S_keywords
    else:  # category == 'G'
        keywords = G_keywords

    # 문장에서 각 키워드의 출현 횟수를 계산합니다.
    keyword_count = 0
    for keyword in keywords:
        if keyword in sentence:
            keyword_count += 1

    return keyword_count

In [30]:
def count_keywords_in_sentence(sentence, category, E_keywords, S_keywords, G_keywords):
    # 카테고리에 따라 적절한 키워드 세트를 선택
    keywords = E_keywords if category == 'E' else S_keywords if category == 'S' else G_keywords

    # 문장에서 키워드 출현 횟수 계산
    return sum(keyword in sentence for keyword in keywords)

# 가장 키워드가 밀집된 ESG 관련 문장 추출 함수
def extract_most_keyword_dense_esg_sentence(text, E_keywords, S_keywords, G_keywords):
    sentences = text.split('.')
    max_keywords = 0
    most_keyword_dense_sentence = ""

    for sentence in sentences:
        if sentence.strip():
            category = find_most_similar_category(sentence)
            if category in ['E', 'S', 'G']:
                keywords_count = count_keywords_in_sentence(sentence, category, E_keywords, S_keywords, G_keywords)
                if keywords_count > max_keywords:
                    max_keywords = keywords_count
                    most_keyword_dense_sentence = sentence.strip()

    return most_keyword_dense_sentence

In [36]:
def add_category_percentages_and_esgce_to_d_sentenf(df, E_keywords, S_keywords, G_keywords):
    for index, row in df.iterrows():
        percentages = calculate_sentence_category_percentages(row['content'])
        df.at[index, 'ESG_Total %'] = percentages['ESG_Total']

        esg_sentence = extract_most_keyword_dense_esg_sentence(row['content'], E_keywords, S_keywords, G_keywords)
        df.at[index, 'ESG_Sentence'] = esg_sentence

    return df

In [123]:
# 새로운 데이터에 대한 예측 수행
df3['감정분석'] = model.predict(df3['content'])
df3['ESG'] = df3['content'].apply(find_most_similar_category)

In [125]:
df3 = add_category_percentages_and_esgce_to_d_sentenf(df3,E_keywords,S_keywords,G_keywords)

In [126]:
df3

,date,기사제목,네이버 뉴스 url,content,감정분석,ESG,ESG_Total %,ESG_Sentence
1919,2021-10-01,[데일리안 오늘뉴스 종합] '유동규 뇌물수수 의혹' 뇌관…민주당 당혹감 속 언급 피...,https://n.news.naver.com/mnews/article/119/000...,더불어민주당 대선 주자인 이재명 경기도지사가 지난 9월 29일 오전 서울 영등포구 ...,긍정,S,62.50,유 전 본부장은 대장동 사업을 기획하고 직접 실무를 담당하는 등 핵심적인 역할을 했...
926,2021-10-01,"연말까지 '마통'닫은 카카오뱅크…토스는 ""억대 대출""",https://n.news.naver.com/mnews/article/422/000...,[앵커] 인터넷전문은행 카카오뱅크가 금융당국의 가계대출 억제 권고에 따라 오늘(1일...,긍정,G,21.05,[앵커] 인터넷전문은행 카카오뱅크가 금융당국의 가계대출 억제 권고에 따라 오늘(1일...
2012,2021-10-01,[전문가에게 듣는다] 방역 당국 “코로나19 예방접종 증명서 불법 사용 시 처벌”,https://n.news.naver.com/mnews/article/056/001...,[앵커] 코로나19 백신 접종률이 오르고 단계적 일상회복 논의가 진행되면서 접종 완...,긍정,G,60.00,"아울러, 다중이용시설에서 위 변조한 증명서를 제시하거나, 부정한 방법으로 사용할 경..."
1180,2021-10-01,2%대 금리 은행 온다...토스뱅크 내주 출범,https://n.news.naver.com/mnews/article/029/000...,10월 5일 사전 신청자 대상 서비스예금·카드·대출 상품 계획 발표업계 최저 2%대...,긍정,G,41.38,10월 5일 사전 신청자 대상 서비스예금·카드·대출 상품 계획 발표업계 최저 2%대...
1684,2021-10-01,[황출새] 최재형 'BTS 등골 빼먹는 文 정부' 논평에 탁현민 '지긋지긋 하다' 外,https://n.news.naver.com/mnews/article/052/000...,[관련 동영상 보기]YTN라디오(FM 94.5) [황보선의 출발 새아침]□ 방송일시...,중립,G,28.28,"◆ 이현웅: 어제 홍남기 경제부총리 겸 기획재정부 장관, 이주열 한국은행 총재, 고..."
...,...,...,...,...,...,...,...,...
61530,2023-09-29,"""매일 커피 대신 주식 투자""…토스 '주식모으기' 1위 종목은",https://n.news.naver.com/mnews/article/003/001...,"장기 재테크 위한 적립식 투자에 관심""주가 조정기, 매입 단가 낮출 수 있어""[서울...",긍정,G,76.92,"토스증권 관계자는 ""2000년대 초반 적립식 펀드 투자 열풍이 불면서 국내 종목 뿐..."
62364,2023-09-29,"추석 연휴, 가볍게 즐길 모바일 게임 5종",https://n.news.naver.com/mnews/article/092/000...,연휴 기간 모바일로 손쉽게 즐길 수 있는 5종 게임 소개올해 추석은 예년과 다르게 ...,긍정,G,54.55,블루 아카이브 대표 이미지진득하게 웰메이드 스토리를 즐기고 싶은 이용자에게는 넥슨 ...
61534,2023-09-30,내 스마트폰에 깔린 뱅킹앱 얼마나 안전할까 [분석+],https://n.news.naver.com/mnews/article/665/000...,더스쿠프 마켓분석 국내 은행 뱅킹앱 보고서두번째 지표➋ 보안성ㆍ가독성서경대 MFS...,긍정,E,75.56,더스쿠프 마켓분석 국내 은행 뱅킹앱 보고서두번째 지표➋ 보안성ㆍ가독성서경대 MFS...
62196,2023-09-30,추석에 받은 용돈…이색 예·적금은 어때?,https://n.news.naver.com/mnews/article/648/000...,인뱅 최대 4%대 고금리 예·적금…청소년층에 큰 인기시중은행들도 MZ 사로잡기…다양...,긍정,G,46.34,이자 받는 정기예금은 가입 즉시 이자가 지급되는 상품으로 가입과 동시에 이자를 출금...


In [127]:
df3.to_csv('토스_감정분석끝.csv', index=False)